# opening a fits

### Há 53 FFIs ao total. Cada FFI possui 84 subfits (85 valores já que um é do header básico). Dentro de cada FFI vou achar o valor de background e modificar a matrizes para se tornarem quadradas. 

In [19]:
#https://archive.stsci.edu/kepler/ffi_display.php
#https://photutils.readthedocs.io/en/stable/background.html

#git fetch --all

#git reset --hard origin/master

#imclean = imclean[:-1,:-1]

In [26]:
# bibliotecas a serem baixadas
%matplotlib inline                           
%config InlineBackend.figure_format='retina'       # Melhora a resolucao do plot
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pylab as plt                     # Permite fazer plots utilizando o pacote matplotlib
plt.rcParams['figure.figsize'] = (18.0, 18.0)      # Determina o tamanho do plot dentro do ipython notebook
from astropy.io import fits, votable               # Permite carregar arquivos fits e votable
from astropy.table import Table                    # Permite carregar tabelas
from astropy import wcs                            # Importa o pacote World Coordinate System
import numpy as np         


#this program will apply the bias subtraction,
#flat fielding and subtract the gradient background,
#it will also align the images to the first image in the list

#if you use this code, please cite Oelkers et al. 2015, AJ, 149, 50
import scipy
from scipy import stats
import scipy.ndimage as ndimage
import astropy
from astropy.io import fits
from astropy.nddata.utils import Cutout2D
from astropy.wcs import WCS
import math
import time

#libraries for image registration
import FITS_tools
from FITS_tools.hcongrid import hcongrid

#import relevant libraries for a list
import glob, os
from os import listdir
from os.path import isfile, join

#import relevant spline libraries
from scipy.interpolate import Rbf

from astropy.stats import biweight_location

In [27]:
#definir se é sonic ou usuario

#user = 'sonic'
user = 'usuario'

In [28]:
#exemplo de como abrir um fits
hdulist = fits.open('/home/' + user + '/Documents/Mestrado/FFIs/8_golden/kplr2009114174833_ffi-cal.fits')
data = hdulist[1].data
#print(data)
print(data.shape)
data = [1,2,3]

print(len(hdulist))
# shd = fits.PrimaryHDU(data, header=hdulist[1].header)
# shd.writeto('/home/' + user + '/Documents/Mestrado/FFIs/8_golden/teste/teste.fits', overwrite = True)
# hdulist[1].header
#from astropy.stats import sigma_clipped_stats
# mean, median, std = sigma_clipped_stats(data, sigma=3.0)
# print((mean, median, std)) 
#from astropy.stats import biweight_location
#print(biweight_location(data)) 
#data[1043]==data[-27]
#hdulist.info()

(1070, 1132)
85


## criar lista de ffis a serem abertos 

### programa em bash
#!/usr/bin/env bash

cd
if [ -a golden.txt ];
	then rm golden.txt
fi

#para printar os files sem os caminhos
#find /home/sonic/Documents/Mestrado/FFIs/8_golden/*.fits -type f -execdir basename '{}' ';'  > /home/sonic/dia_kepler/mychanges/Python/golden.txt

#para printar os files com os caminhos
find /home/sonic/Documents/Mestrado/FFIs/8_golden/*.fits -type f > /home/sonic/dia_kepler/mychanges/Python/golden.txt

find /home/sonic/Documents/Mestrado/FFIs/*.fits -type f > /home/sonic/dia_kepler/mychanges/Python/allffis.txt

In [29]:
arquivos = !find *.txt
arquivos
arq=[]
for l in arquivos:
    with open(l) as f:
        arq.append(f.read().splitlines())
for y in range(len(arq)):
     arq[y] = np.sort(arq[y])
allffis = arq[0]
golden = arq[1]
golden
golden[0].split('/')[-1]
golden

array(['/home/usuario/Documents/Mestrado/FFIs/8_golden/kplr2009114174833_ffi-cal.fits',
       '/home/usuario/Documents/Mestrado/FFIs/8_golden/kplr2009114204835_ffi-cal.fits',
       '/home/usuario/Documents/Mestrado/FFIs/8_golden/kplr2009115002613_ffi-cal.fits',
       '/home/usuario/Documents/Mestrado/FFIs/8_golden/kplr2009115053616_ffi-cal.fits',
       '/home/usuario/Documents/Mestrado/FFIs/8_golden/kplr2009115080620_ffi-cal.fits',
       '/home/usuario/Documents/Mestrado/FFIs/8_golden/kplr2009115131122_ffi-cal.fits',
       '/home/usuario/Documents/Mestrado/FFIs/8_golden/kplr2009115173611_ffi-cal.fits',
       '/home/usuario/Documents/Mestrado/FFIs/8_golden/kplr2009116035924_ffi-cal.fits'],
      dtype='<U77')

## Achando background

### Kepler CCD
<img src="CCD_blockdiagram1.jpg">
### Tess CCD
<img src="./tess_ccd.png">

In [34]:
# Rodar apenas uma vez
## consertando as matrizes - cortando linhas e colunas ruins + transformando em matriz quadrada

for i in golden:
    hdulist = fits.open(i)
    for j in range(1,len(hdulist)):#      cada uma das 84 imagens
        data = hdulist[j].data
        sky = biweight_location(data)
        data2 = data[20:1044,12:1112]
#devo 
#         shd = fits.PrimaryHDU(data, header=hdulist[j].header) #esse e o de baixo será esta posição?
#     shd.writeto('/home/' + user + '/Documents/Mestrado/FFIs/8_golden/teste/'+ i.split('/')[-1]+'.fits', overwrite = True)
#         hdulist[1].header
print(data2[0,1100])

IndexError: index 1100 is out of bounds for axis 1 with size 1100

In [ ]:
data.shape

In [ ]:
from photutils.datasets import make_100gaussians_image
data = make_100gaussians_image()

import matplotlib.pyplot as plt
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, norm=norm, origin='lower', cmap='Greys_r')

In [ ]:
#####UPDATE INFORMATION HERE####
#DO YOU WANT TO FLAT FIELD AND BIAS SUBTRACT?
biassub = 0 # yes = 1 no = 0 to bias subtract
flatdiv = 0 # yes = 1 no = 0 to flat field
align = 1# yes = 1 no = 0 to align based on coordinates

#useful directories
rawdir = '/home/usuario/Documents/Mestrado/FFIs/8_golden/' #directory with the raw images
cdedir = '/home/usuario/dia_kepler/mychanges/Python' #directory where the code 'lives'
caldir = 'N/A' #directory with the calibration images such as bias & flat
clndir = '/home/usuario/Documents/Mestrado/FFIs/8_golden/clean/'#directory for the cleaned images to be output